In [ ]:
!pip install torch diffusers transformers pillow
!pip3 install ipywidgets

## SD3.5 Large

In [ ]:
import torch
from diffusers import StableDiffusion3Pipeline
from PIL import Image
import os

# Define custom cache directory for model downloads
model_cache_dir = "sd" 

# Define custom directory to save generated images
output_image_path = "sd3.5L_image.png"

# Create the directories if they don't exist
os.makedirs(model_cache_dir, exist_ok=True)

# Load the Stable Diffusion model with a specified cache directory
pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3.5-large", torch_dtype=torch.bfloat16,cache_dir=model_cache_dir)

pipe.to("cuda")  # Move the model to GPU for faster inference

# Define your prompt (the text description of the image you want to generate)
prompt = "A futuristic city at sunset, with flying cars and neon lights"

# Generate the image
with torch.no_grad():
    image = pipe(prompt,num_inference_steps=28,guidance_scale=3.5).images[0]

# Save the generated image to the specified path
image.save(output_image_path)

# Optionally, display the image
image.show()

print(f"Image saved to {output_image_path}")


## Quantize SD 3.5 Large

In [ ]:
from diffusers import BitsAndBytesConfig, SD3Transformer2DModel
import torch

# Define the model ID and cache directory
model_cache_dir = "sd/sd3.5lar"
model_id = "stabilityai/stable-diffusion-3.5-large"

# Quantization configuration (nf4)
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    cache_dir=model_cache_dir
)

# Load the model with the quantization config
model_nf4 = SD3Transformer2DModel.from_pretrained(
    model_id,
    subfolder="transformer",
    quantization_config=nf4_config,
    torch_dtype=torch.float16,
    cache_dir=model_cache_dir
)

# Save the quantized model to the "quantized_model" directory
quantized_model_dir = "quantized_model"
model_nf4.save_pretrained(quantized_model_dir)

print(f"Quantized model saved in {quantized_model_dir}")


## Running Quantized SD 3.5 Large

In [ ]:
from diffusers import StableDiffusion3Pipeline, SD3Transformer2DModel
import torch

# Define the model ID and cache directory
model_cache_dir = "sd"
model_id = "stabilityai/stable-diffusion-3.5-large"

# Load the quantized model from the 'quantized_model' directory
quantized_model_dir = "quantized_model/sd3.5lar"
model_nf4 = SD3Transformer2DModel.from_pretrained(
    quantized_model_dir,
    torch_dtype=torch.float16,
    cache_dir=model_cache_dir
)

# Load the pipeline using the quantized model
pipeline = StableDiffusion3Pipeline.from_pretrained(
    model_id,
    transformer=model_nf4,
    torch_dtype=torch.float16,
    cache_dir=model_cache_dir
)
pipeline.enable_model_cpu_offload()

# Define the prompt for image generation
prompt = "anime red hair girl, with yellow eyes, in garden"

# Generate the image
image = pipeline(
    prompt=prompt,
    num_inference_steps=28,
    guidance_scale=4.5,
    max_sequence_length=512,
).images[0]

# Save the generated image
image.save("generated_image_from_quantized_model.png")
print("Image generated and saved as 'generated_image_from_quantized_model.png'")
image.show()